<a href="https://colab.research.google.com/github/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm/blob/oscar2/notebooks/oscar_test_config_kernel_size.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# reference: https://www.programiz.com/python-programming/datetime/current-time

from datetime import datetime
import pytz

print(datetime.now(pytz.timezone('America/New_York')).strftime('%m/%d(%a)_%H:%M:%S'))

11/18(Fri)_14:38:10


# Wandb Install, Login, Import

In [2]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!wandb login "6f19b1e6735ebc69af24f18d5b426262416027fb"

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
import wandb

# Torchsummary Install, Import

In [81]:
!pip install torch-summary==1.4.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [82]:
from torchsummary import summary

summary

<function torchsummary.torchsummary.summary(model: torch.nn.modules.module.Module, input_data: Union[torch.Tensor, torch.Size, Sequence[torch.Tensor], Sequence[Union[int, Sequence[Any], torch.Size]], NoneType] = None, *args: Any, batch_dim: Union[int, NoneType] = 0, branching: bool = True, col_names: Union[Iterable[str], NoneType] = None, col_width: int = 25, depth: int = 3, device: Union[torch.device, NoneType] = None, dtypes: Union[List[torch.dtype], NoneType] = None, verbose: int = 1, **kwargs: Any) -> torchsummary.model_statistics.ModelStatistics>

# Clone Team's Code

In [5]:
!rm -r /content/csgy6953_DeepLearning_Midterm/

In [6]:
!git clone -b config_kernel_size_on_updated_main "https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"

Cloning into 'csgy6953_DeepLearning_Midterm'...
remote: Enumerating objects: 614, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 614 (delta 42), reused 39 (delta 31), pack-reused 507
Receiving objects: 100% (614/614), 157.10 KiB | 1.45 MiB/s, done.
Resolving deltas: 100% (393/393), done.


In [7]:
!cp -r /content/csgy6953_DeepLearning_Midterm/src/ .

In [8]:
!cat src/model.py

from enum import Enum
from typing import List, Optional, Tuple

import torch
import torch.nn as nn


class ResidualBlockType(Enum):
    """
    Enum class to represent the residual block type for ResNet
    """

    BASIC = 0
    BOTTLENECK = 1


class LayerType(Enum):
    """
    Enum class to represent layer within for ResidualBlock and for BottleneckResidualBlock
    """

    # Disambiguation: here "layer" refers to the individual layer within a block,
    # not a "residual layer" containing one or more blocks
    CONV = 0


class LayerLoc(Enum):
    """
    Enum class to represent a layer's location within a block
    """

    MAIN_BLOCK_CONV1 = 0
    MAIN_BLOCK_CONV2 = 1
    MAIN_BLOCK_CONV3 = 2

    SHORTCUT_IDENTITY = 6   # identity
    SHORTCUT_CONV_STEM = 7


def generate_layer(
    block_type: ResidualBlockType,
    layer_type: LayerType, 
    layer_loc: LayerLoc, # position of this layer within the block starting from index 1 
    num_channels: int,
    main_block_kernel_siz

# Import, Seed, Device

In [9]:
import torch
import torch.nn as nn

import time
import random

In [10]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Load Data

In [62]:
from src.data import get_transformed_data, make_data_loaders
from src.transforms import make_auto_transforms # used to use: make_transforms

BATCH_SIZE = 128
VALID_RATIO = 0.1

train_data, valid_data, test_data = \
get_transformed_data(make_transforms=make_auto_transforms, valid_ratio=VALID_RATIO)

train_iter, valid_iter, test_iter = \
make_data_loaders(train_data, valid_data, test_data, BATCH_SIZE)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Model

In [63]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# from typing import List, Tuple, Optional


# class ResidualBlock(nn.Module):
#     '''
#     Class representing a convolutional residual block 
#     '''

#     def __init__(self, num_channels: int, use_stem: bool = False, strides: int = 1, dropout: Optional[float] = None, kernel_size: int = 3):
#         '''
#         Creates a new instance of a Residual Block
#         @param: num_channels (int) - the number of output channels for all convolutions in 
#             the block
#         @param: use_stem (bool) - whether a 1x1 convolution is needed to downsample the
#             residual
#         @param: strides (int) - the number of strides to use in the convolutions, defaults to 1
#         @param: dropout (float) - if present, adds a dropout between the hidden layers
#         '''
#         super().__init__()
#         self.num_channels = num_channels
#         self.use_stem = use_stem
#         self.strides = strides

#         self.kernel_size = kernel_size

#         self.dropout = nn.Dropout(dropout) if dropout is not None else None

#         if self.kernel_size == 3:
#             self.conv1 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1, stride=strides)
#             self.conv2 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1)

#         # partial solution 1:
#         if kernel_size == 2:
#             self.conv1 = nn.LazyConv2d(num_channels, kernel_size=2, padding=1, stride=strides)
#             self.conv2 = nn.LazyConv2d(num_channels, kernel_size=2, padding=0)
#             # fatal issue: when input is [256,1,1], MUST pad in order to apply 2x2 kernel

#         # # partial solution 2:
#         # if self.kernel_size == 2:
#         #     self.conv1 = nn.LazyConv2d(num_channels, kernel_size=2, padding=1, stride=strides)
#         #     self.conv2 = nn.LazyConv2d(num_channels, kernel_size=2, padding=1)

#         self.relu = nn.ReLU(inplace=True)
#         self.out = nn.ReLU(inplace=True)
#         self.bn1 = nn.LazyBatchNorm2d()
#         self.bn2 = nn.LazyBatchNorm2d()

#         self.conv_stem = None
#         if use_stem:

#             if kernel_size == 3:
#                 self.conv_stem = nn.LazyConv2d(num_channels, kernel_size=1, stride=strides)
#             if kernel_size == 2:
#                 self.conv_stem = nn.LazyConv2d(num_channels, kernel_size=1, stride=strides)

#     def forward(self, inputs: torch.Tensor) -> torch.Tensor:
#         print()
#         print('--------ResidualBlock:--------')
#         print(f'(use_stem={self.use_stem})')
#         print('block input:', inputs.shape)

#         shortcut = inputs

#         print()
#         print('F(x):', self.conv1)
#         x = self.relu(self.bn1(self.conv1(inputs)))
#         print('output:', x.shape)

#         if self.dropout is not None:
#             x = self.dropout(x)
        
#         print()
#         print('F(x):', self.conv2)
#         x = self.bn2(self.conv2(x))
#         print('output:', x.shape)

#         if self.use_stem:
#             # downsample skip connection
#             print()
#             print('S(x):', self.conv_stem)
#             shortcut = self.conv_stem(shortcut)
#             print('output:', shortcut.shape)

#         # partial solution 2:
#         # if self.kernel_size == 2:
#         #     if x.shape[-1] > shortcut.shape[-1]:
#         #         x = F.pad(x, pad = (-1, -1, -1, -1))
#         #         print('negative padding =>', x.shape)
#         #         # fatal error: when F(x)'s output becomes [512,2,2] whiel input was [256,1,1], this would reduce F(x)'s output to [512,0,0]
 
#         # add in skip connection
#         x += shortcut

#         return self.out(x)


# class StemConfig:
#     '''
#     convenience class to encapsulate configuration options
#     for the ResNet stem
#     '''

#     def __init__(self, num_channels, kernel_size, stride, padding):
#         self.num_channels = num_channels
#         self.kernel_size = kernel_size
#         self.stride = stride
#         self.padding = padding


# class ResNet(nn.Module):
#     '''
#     Class representing a full ResNet model
#     '''

#     def __init__(self, architecture: List[Tuple[int, int, float]], stem_config: Optional[StemConfig], output_size: int = 10, *args, **kwargs):
#         '''
#         returns an instance of a ResNet
#         '''
#         super().__init__()
#         if stem_config is not None:
#             self.stem = self.create_stem(
#                 stem_config.num_channels,
#                 stem_config.kernel_size,
#                 stem_config.stride,
#                 stem_config.padding
#             )
#         else:
#             self.stem = self.create_stem()
#         self.classifier = self.create_classifier(output_size)

#         self.body = nn.Sequential()
#         for idx, block_def in enumerate(architecture):
#             self.body.add_module(
#                 f"block_{idx+2}", self.create_block(*block_def, first_block=(idx == 0)))

#     def forward(self, inputs: torch.Tensor) -> torch.Tensor:
#         """
#         Performs forward pass of the inputs through the network
#         """
#         x = self.stem(inputs)
#         x = self.body(x)
#         return self.classifier(x)

#     def create_stem(self, num_channels: int = 64, kernel_size: int = 7, stride: int = 2, padding: int = 3) \
#             -> nn.Sequential:
#         """
#         Creates a sequential stem as the first component of the model
#         """
#         return nn.Sequential(
#             nn.LazyConv2d(num_channels, kernel_size=kernel_size,
#                           padding=padding, stride=stride),
#             nn.LazyBatchNorm2d(),
#             nn.ReLU(inplace=True),
#             # nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
#         )

#     def create_classifier(self, num_classes: int) -> nn.Sequential:
#         '''
#         Creates a sequential classifier head at the very 
#         '''
#         return nn.Sequential(
#             nn.AdaptiveAvgPool2d(1),
#             nn.Flatten(),
#             nn.LazyLinear(num_classes)
#         )

#     def create_block(self, num_residuals: int, num_channels: int, dropout: float, kernel_size: int, first_block: bool = False) -> nn.Sequential:
#         layer = []
#         for i in range(num_residuals):
#             if i == 0 and not first_block:
#                 layer.append(ResidualBlock(num_channels, dropout=dropout, kernel_size=kernel_size, use_stem=True, strides=2))
#             else:
#                 layer.append(ResidualBlock(num_channels, dropout=dropout, kernel_size=kernel_size))
#         return nn.Sequential(*layer)


In [64]:
# from src.model import StemConfig

# # 1 tuple == 1 layer
# # how many blocks in each layer, out_channels, dropout prob, kernel_size
# architecture = [
#     (2, 64, 0.5, 2),
#     (2, 128, 0.5, 2),
#     (2, 256, 0.5, 2),
#     (2, 512, 0.5, 2),
# ]

In [65]:
from enum import Enum
from typing import List, Optional, Tuple

import torch
import torch.nn as nn


class ResidualBlockType(Enum):
    """
    Enum class to represent the residual block type for ResNet
    """

    BASIC = 0
    BOTTLENECK = 1


class LayerType(Enum):
    """
    Enum class to represent layer within for ResidualBlock and for BottleneckResidualBlock
    """

    # Disambiguation: here "layer" refers to the individual layer within a block,
    # not a "residual layer" containing one or more blocks
    CONV = 0


class LayerLoc(Enum):
    """
    Enum class to represent a layer's location within a block
    """

    MAIN_BLOCK_CONV1 = 0
    MAIN_BLOCK_CONV2 = 1
    MAIN_BLOCK_CONV3 = 2

    SHORTCUT_IDENTITY = 6   # identity
    SHORTCUT_CONV_STEM = 7


def generate_layer(
    block_type: ResidualBlockType,
    layer_type: LayerType, 
    layer_loc: LayerLoc, # position of this layer within the block starting from index 1 
    num_channels: int,
    main_block_kernel_size: int,
    strides: int = 1,
    factor: int = 4,
    use_bias: bool = False,
):
    """
    Returns a layer with the most appropriate parameters such as padding 
    """
    if block_type == ResidualBlockType.BASIC:
        if layer_type == LayerType.CONV:

            if main_block_kernel_size == 3:
                layer_locator = {
                    LayerLoc.MAIN_BLOCK_CONV1: nn.LazyConv2d(
                        num_channels,
                        kernel_size=3, padding=1, # ResidualBlock.conv1
                        stride=strides, bias=use_bias
                        ), 
                     LayerLoc.MAIN_BLOCK_CONV2: nn.LazyConv2d(
                        num_channels,
                        kernel_size=3, padding=1, # ResidualBlock.conv2
                        bias=use_bias
                        ),
                     LayerLoc.SHORTCUT_IDENTITY: nn.Identity(),
                     LayerLoc.SHORTCUT_CONV_STEM: nn.LazyConv2d(
                        num_channels, 
                        kernel_size=1, stride=strides, # ResidualBlock.conv_stem
                        bias=use_bias
                        )
                    }
            
            # partial solution 1:
            # fatal issue: when input is [256,1,1], MUST pad in order to apply 2x2 kernel
            if main_block_kernel_size == 2:
                layer_locator = {
                    LayerLoc.MAIN_BLOCK_CONV1: nn.LazyConv2d(
                        num_channels,
                        kernel_size=2, padding=1, # ResidualBlock.conv1
                        stride=strides, bias=use_bias
                        ), 
                     LayerLoc.MAIN_BLOCK_CONV2: nn.LazyConv2d(
                        num_channels,
                        kernel_size=2, padding=0, # ResidualBlock.conv2
                        bias=use_bias
                        ),
                     LayerLoc.SHORTCUT_IDENTITY: nn.Identity(),
                     LayerLoc.SHORTCUT_CONV_STEM: nn.LazyConv2d(
                        num_channels, 
                        kernel_size=1, stride=strides, # ResidualBlock.conv_stem
                        bias=use_bias
                        )
                    }
            
    if block_type == ResidualBlockType.BOTTLENECK:
        if layer_type == LayerType.CONV:

            if main_block_kernel_size == 3:
                layer_locator = {
                    LayerLoc.MAIN_BLOCK_CONV1: nn.LazyConv2d(
                        num_channels // factor,
                        kernel_size=1, padding=0, # BottleneckResidualBlock.conv1
                        bias=use_bias
                        ), 
                    LayerLoc.MAIN_BLOCK_CONV2: nn.LazyConv2d(
                        num_channels // factor,
                        kernel_size=3, padding=1, stride=strides, # BottleneckResidualBlock.conv2
                        bias=use_bias
                        ),
                    LayerLoc.MAIN_BLOCK_CONV3: nn.LazyConv2d(
                        num_channels, 
                        kernel_size=1, padding=0, # BottleneckResidualBlock.conv3
                        bias=use_bias
                        ),
                    LayerLoc.SHORTCUT_IDENTITY: nn.Identity(),
                    LayerLoc.SHORTCUT_CONV_STEM: nn.LazyConv2d(
                        num_channels, 
                        kernel_size=1, stride=strides, # BottleneckResidualBlock.conv_stem
                        bias=use_bias
                        )
                    }
    return layer_locator[layer_loc]
    

class ResidualBlock(nn.Module):
    """
    Class representing a convolutional residual block
    """

    def __init__(
        self,
        num_channels: int,
        use_stem: bool = False,
        strides: int = 1,
        dropout: Optional[float] = None,
        use_bias: bool = False,
        main_block_kernel_size: int = 3
    ):
        """
        Creates a new instance of a Residual Block
        @param: num_channels (int) - the number of output channels for all convolutions in
            the block
        @param: use_stem (bool) - whether a 1x1 convolution is needed to downsample the
            residual
        @param: strides (int) - the number of strides to use in the convolutions, defaults to 1
        @param: dropout (float) - if present, adds a dropout between the hidden layers
        """
        super().__init__()
        self.num_channels = num_channels
        self.use_stem = use_stem
        self.strides = strides

        self.dropout = nn.Dropout(dropout) if dropout is not None else None
        self.conv1 = generate_layer(
            block_type = ResidualBlockType.BASIC,
            layer_type = LayerType.CONV,
            layer_loc = LayerLoc.MAIN_BLOCK_CONV1,
            num_channels = num_channels,
            main_block_kernel_size = main_block_kernel_size,
            strides = strides,
            use_bias = use_bias,
        )
        self.conv2 = generate_layer(
            block_type = ResidualBlockType.BASIC,
            layer_type = LayerType.CONV,
            layer_loc = LayerLoc.MAIN_BLOCK_CONV2,
            num_channels = num_channels,
            main_block_kernel_size = main_block_kernel_size,
            use_bias = use_bias,
        )
        self.relu = nn.ReLU(inplace=True)
        self.out = nn.ReLU(inplace=True)
        self.bn1 = nn.LazyBatchNorm2d()
        self.bn2 = nn.LazyBatchNorm2d()

        self.identity = None
        self.conv_stem = None
        if use_stem:
            self.conv_stem = generate_layer(
                block_type = ResidualBlockType.BASIC,
                layer_type = LayerType.CONV,
                layer_loc = LayerLoc.SHORTCUT_CONV_STEM,
                num_channels = num_channels, 
                main_block_kernel_size = main_block_kernel_size,
                strides = strides,
                use_bias = use_bias
            )
        else:
            self.identity = generate_layer(
                block_type = ResidualBlockType.BASIC,
                layer_type = LayerType.CONV,
                layer_loc = LayerLoc.SHORTCUT_IDENTITY,
                num_channels = num_channels,
                main_block_kernel_size = main_block_kernel_size
            )

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        shortcut = inputs

        x = self.relu(self.bn1(self.conv1(inputs)))
        if self.dropout is not None:
            x = self.dropout(x)
        x = self.bn2(self.conv2(x))
        
        if self.use_stem:
            # downsample skip connection
            shortcut = self.conv_stem(shortcut)
        else:
            shortcut = self.identity(shortcut)

        # add in skip connection
        x += shortcut
        return self.out(x)


class BottleneckResidualBlock(nn.Module):
    """
    Class representing a convolutional residual block with a bottleneck
    This class was built with reference to:
    https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py
    """

    def __init__(
        self,
        num_channels: int,
        use_stem: bool = False,
        strides: int = 1,
        factor: int = 4,
        dropout: Optional[float] = None,
        use_bias: bool = False,
    ):
        """
        Creates a new instance of a Residual BottleNeck Block
        @param: num_channels (int) - the number of output channels for all convolutions in the block
        @param: use_stem (bool) - whether a 1x1 convolution is needed to downsample the residual
        @param: strides (int) - the number of strides to use in the convolutions, defaults to 1
        @param: factor (int) - the factor by which the input channels will be reduced for the bottleneck
        @param: dropout (float) - if present, adds a dropout between the hidden layers
        """
        super().__init__()
        self.num_channels = num_channels
        self.use_stem = use_stem
        self.strides = strides
        self.factor = factor
        self.dropout1 = nn.Dropout(dropout) if dropout is not None else None
        self.dropout2 = nn.Dropout(dropout) if dropout is not None else None

        # First convolutional layer with normalization
        self.conv1 = nn.LazyConv2d(
            num_channels // factor, kernel_size=1, padding=0, bias=use_bias
        )
        self.bn1 = nn.LazyBatchNorm2d()

        # Second convolutional layer with normalization
        self.conv2 = nn.LazyConv2d(
            num_channels // factor,
            kernel_size=3,
            padding=1,
            stride=strides,
            bias=use_bias,
        )
        self.bn2 = nn.LazyBatchNorm2d()

        # Third convolutional layer with normalization
        self.conv3 = nn.LazyConv2d(
            num_channels, kernel_size=1, padding=0, bias=use_bias
        )
        self.bn3 = nn.LazyBatchNorm2d()

        self.relu = nn.ReLU(inplace=True)

        self.conv_stem = None
        if use_stem:
            # Bottleneck residual block
            self.conv_stem = nn.LazyConv2d(
                num_channels, kernel_size=1, stride=strides, bias=use_bias
            )

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        shortcut = inputs
        x = self.relu(self.bn1(self.conv1(inputs)))
        if self.dropout1 is not None:
            x = self.dropout1(x)
        x = self.relu(self.bn2(self.conv2(x)))
        if self.dropout2 is not None:
            x = self.dropout2(x)
        x = self.bn3(self.conv3(x))
        if self.use_stem:
            # downsample skip connection
            shortcut = self.conv_stem(shortcut)

        # add in skip connection
        x += shortcut
        return self.relu(x)


class StemConfig:
    """
    convenience class to encapsulate configuration options
    for the ResNet stem
    """

    def __init__(self, num_channels, kernel_size, stride, padding):
        self.num_channels = num_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding


def generate_block(
    block_type: ResidualBlockType,
    num_channels: int,
    use_stem: bool = False,
    strides: int = 1,
    factor: int = 4,
    dropout: Optional[float] = None,
    use_bias: bool = False,
    main_block_kernel_size: int = 3
):
    """
    Returns either a Residual Block or a ResidualBottleneck
    """
    if block_type == ResidualBlockType.BASIC:
        return ResidualBlock(
            num_channels,
            use_stem=use_stem,
            strides=strides,
            dropout=dropout,
            use_bias=use_bias,
            main_block_kernel_size=main_block_kernel_size
        )
    else:
        return BottleneckResidualBlock(
            num_channels,
            use_stem=use_stem,
            strides=strides,
            factor=factor,
            dropout=dropout,
            use_bias=use_bias,
            main_block_kernel_size=main_block_kernel_size
        )


class ResNet(nn.Module):
    """
    Class representing a full ResNet model
    """

    def __init__(
        self,
        architecture: List[Tuple[ResidualBlockType, int, int, float, int]],
        stem_config: Optional[StemConfig],
        output_size: int = 10,
        use_bias: bool = False,
        *args,
        **kwargs,
    ):
        """
        returns an instance of a ResNet
        """
        super().__init__()
        self.use_bias = use_bias
        if stem_config is not None:
            self.stem = self.create_stem(
                stem_config.num_channels,
                stem_config.kernel_size,
                stem_config.stride,
                stem_config.padding,
                use_bias=use_bias,
            )
        else:
            self.stem = self.create_stem(use_bias=use_bias)
        self.classifier = self.create_classifier(output_size, use_bias=use_bias)

        self.body = nn.Sequential()
        for idx, block_def in enumerate(architecture):
            self.body.add_module(
                f"block_{idx+2}",
                self.create_block(
                    *block_def, first_block=(idx == 0), use_bias=use_bias
                ),
            )

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        """
        Performs forward pass of the inputs through the network
        """
        x = self.stem(inputs)
        x = self.body(x)
        return self.classifier(x)

    def create_stem(
        self,
        num_channels: int = 64,
        kernel_size: int = 7,
        stride: int = 2,
        padding: int = 3,
        use_bias: bool = False,
    ) -> nn.Sequential:
        """
        Creates a sequential stem as the first component of the model
        """
        return nn.Sequential(
            nn.LazyConv2d(
                num_channels,
                kernel_size=kernel_size,
                padding=padding,
                stride=stride,
                bias=use_bias,
            ),
            nn.LazyBatchNorm2d(),
            nn.ReLU(inplace=True),
        )

    def create_classifier(
        self, num_classes: int, use_bias: bool = False
    ) -> nn.Sequential:
        """
        Creates a sequential classifier head at the very
        """
        return nn.Sequential(
            nn.AdaptiveAvgPool2d(1), nn.Flatten(), nn.LazyLinear(num_classes)
        )

    def create_block(
        self,
        block_type: ResidualBlockType,
        num_residuals: int,
        num_channels: int,
        dropout: Optional[float] = None,
        main_block_kernel_size: int = 3,
        first_block: bool = False,
        use_bias: bool = False,
    ) -> nn.Sequential:
        """
        Given our inputs, generates either a ResidualBlock or ResidualBottleNeck and addes it to our
        sequence of layers
        """
        layer = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                layer.append(
                    generate_block(
                        block_type,
                        num_channels,
                        use_stem=True,
                        strides=2,
                        dropout=dropout,
                        use_bias=use_bias,
                        main_block_kernel_size=main_block_kernel_size
                    )
                )
            else:
                layer.append(
                    generate_block(
                        block_type, num_channels, dropout=dropout, use_bias=use_bias, 
                        main_block_kernel_size=main_block_kernel_size
                    )
                )
        return nn.Sequential(*layer)


In [66]:
from src.model import StemConfig, ResidualBlockType # ResNet

MAIN_BLOCK_KERNEL_SIZE = 2

# 1 tuple == 1 layer
# block with or without bottleneck, how many blocks in each layer, out_channels, dropout prob, kernel_size
architecture = [
    (ResidualBlockType.BASIC, 2,  64, 0.5, MAIN_BLOCK_KERNEL_SIZE),
    (ResidualBlockType.BASIC, 2, 128, 0.5, MAIN_BLOCK_KERNEL_SIZE),
    (ResidualBlockType.BASIC, 2, 256, 0.5, MAIN_BLOCK_KERNEL_SIZE),
    (ResidualBlockType.BASIC, 2, 512, 0.5, MAIN_BLOCK_KERNEL_SIZE),
]

In [67]:
for temp_idx, temp_block_def in enumerate(architecture):
    print(temp_idx, temp_block_def)

0 (<ResidualBlockType.BASIC: 0>, 2, 64, 0.5, 2)
1 (<ResidualBlockType.BASIC: 0>, 2, 128, 0.5, 2)
2 (<ResidualBlockType.BASIC: 0>, 2, 256, 0.5, 2)
3 (<ResidualBlockType.BASIC: 0>, 2, 512, 0.5, 2)


In [68]:
config = StemConfig(num_channels=64, kernel_size=3, stride=1, padding=1)

model  = ResNet(architecture, stem_config=config, output_size=10)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [69]:
from src.utils import count_parameters

## Initialize model shapes

In [70]:
# intialize a new model

# inputs = torch.empty((BATCH_SIZE, 3, 512, 512)) # passed
inputs = torch.empty((BATCH_SIZE, 3, 32, 32)) #  passed
# inputs = torch.empty((BATCH_SIZE, 3, 4, 4))

inputs.normal_()

model = model.to(device)

outputs = model(inputs.to(device)) 
# (Oscar) observation: 2022.11/13(7)_a08.14: internally, this converts all nn.LazyConv2d layers to nn.Conv2d
# to see this, run print(model) both before and after this operation

print('-------------------')
print('-------------------')

print(model)
print(count_parameters(model))
print(outputs.size())

-------------------
-------------------
ResNet(
  (stem): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (classifier): Sequential(
    (0): AdaptiveAvgPool2d(output_size=1)
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
  (body): Sequential(
    (block_2): Sequential(
      (0): ResidualBlock(
        (dropout): Dropout(p=0.5, inplace=False)
        (conv1): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
        (conv2): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1), bias=False)
        (relu): ReLU(inplace=True)
        (out): ReLU(inplace=True)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_s

## Count parameters

In [71]:
num_parameters, num_parameters_requiring_grad = count_parameters(model)
print(num_parameters)

5069130


In [84]:
QUIET_VERBOSE = 0

one_sample_input_shape = (3, 32, 32)

summary(model, one_sample_input_shape, verbose = QUIET_VERBOSE)

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 32, 32]          --
|    └─Conv2d: 2-1                       [-1, 64, 32, 32]          1,728
|    └─BatchNorm2d: 2-2                  [-1, 64, 32, 32]          128
|    └─ReLU: 2-3                         [-1, 64, 32, 32]          --
├─Sequential: 1-2                        [-1, 512, 4, 4]           --
|    └─Sequential: 2-4                   [-1, 64, 32, 32]          --
|    |    └─ResidualBlock: 3-1           [-1, 64, 32, 32]          33,024
|    |    └─ResidualBlock: 3-2           [-1, 64, 32, 32]          33,024
|    └─Sequential: 2-5                   [-1, 128, 16, 16]         --
|    |    └─ResidualBlock: 3-3           [-1, 128, 16, 16]         107,008
|    |    └─ResidualBlock: 3-4           [-1, 128, 16, 16]         131,584
|    └─Sequential: 2-6                   [-1, 256, 8, 8]           --
|    |    └─ResidualBlock: 3-5           [-1, 256, 8, 8]       

## Initialize parameters

In [72]:
from src.utils import initialize_parameters

# initialize model weights
model.apply(initialize_parameters);

# Train, validate, log

In [73]:
from src.engine import train_one_epoch, evaluate

In [74]:
EPOCHS  = 10

learning_rate = 1e-3

In [75]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [76]:
## setup wandb logging

assert 'wandb' in locals() # 'wandb' has been imported already

ENTITY_NAME  = 'dlf22_mini_project' # this is our team name
PROJECT_NAME = 'Junk_Test_Project'
RUN_NAME     = 'resnet_osca_kernel_size_2_2022.11.18.p02.59'

WANDB_CONFIG = \
{"architecture"  : architecture,    # the model
 "num_parameters": num_parameters,  # the model
 "learning_rate" : learning_rate,   # how to gradient descent
 "batch_size"    : BATCH_SIZE,      # how to gradient descent
 "epochs"        : EPOCHS,}         # train for how long

wandb\
.init(name=RUN_NAME, project=PROJECT_NAME, entity=ENTITY_NAME,config=WANDB_CONFIG)

In [77]:
# save to disk the "best" model === the model with the lowest validation loss
best_model_path = RUN_NAME + '.pt'

In [80]:
from src.utils import epoch_time

best_loss = float('inf')
for epoch in range(11, EPOCHS+10+1):

    print(f"Epoch {epoch}")
    
    ## TRAIN
    start = time.time()
    # train
    train_loss, train_acc  = train_one_epoch(model, train_iter, criterion, optimizer, device)
    # log & echo
    train_mins, train_secs = epoch_time(start, time.time())
    wandb.log({"train_loss": train_loss,
               "train_acc" : train_acc,
               "epoch"     : epoch})
    print(f"\tTrain elapsed: {train_mins}:{train_secs}, loss: {train_loss:.4f}, acc: {train_acc * 100:.2f}%")

    ## VALIDATE
    start = time.time()
    # validate
    val_loss, val_acc  = evaluate(model, valid_iter, criterion, device)
    # log & echo   
    val_mins, val_secs = epoch_time(start, time.time())
    wandb.log({"val_loss": val_loss,
               "val_acc" : val_acc,
               "epoch"   : epoch,})
    print(f"\tValidation elapsed: {val_mins}:{val_secs}, loss: {val_loss:.4f}, acc: {val_acc * 100:.2f}%")

    ## Memorize "best" model === the model with the lowest validation loss
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), best_model_path)

Epoch 11
	Train elapsed: 0:59, loss: 0.7599, acc: 73.29%
	Validation elapsed: 0:1, loss: 0.5800, acc: 80.51%
Epoch 12
	Train elapsed: 1:0, loss: 0.7153, acc: 75.08%
	Validation elapsed: 0:1, loss: 0.5992, acc: 80.10%
Epoch 13
	Train elapsed: 1:0, loss: 0.6907, acc: 75.89%
	Validation elapsed: 0:1, loss: 0.5318, acc: 82.19%
Epoch 14
	Train elapsed: 1:0, loss: 0.6645, acc: 76.69%
	Validation elapsed: 0:1, loss: 0.5305, acc: 82.29%
Epoch 15
	Train elapsed: 1:0, loss: 0.6330, acc: 77.98%
	Validation elapsed: 0:1, loss: 0.5245, acc: 82.05%
Epoch 16
	Train elapsed: 1:0, loss: 0.6102, acc: 78.62%
	Validation elapsed: 0:1, loss: 0.4962, acc: 83.87%
Epoch 17
	Train elapsed: 0:59, loss: 0.5885, acc: 79.42%
	Validation elapsed: 0:1, loss: 0.4883, acc: 83.75%
Epoch 18
	Train elapsed: 0:59, loss: 0.5665, acc: 80.21%
	Validation elapsed: 0:1, loss: 0.5026, acc: 83.79%
Epoch 19
	Train elapsed: 1:0, loss: 0.5480, acc: 80.81%
	Validation elapsed: 0:1, loss: 0.4819, acc: 84.36%
Epoch 20
	Train elapsed: 